# Elevation Interpolation

In [1]:
# import libraries
import arcpy
from arcpy import env # to set working environmnet
import pandas as pd
import psycopg2
import os # file path managment

### Set up environmnet

In [2]:
# Get CWD

cwd = _dh[0] #this is a universal variable that calls the directory where the .ipynb file is located


# Link it to the QAQC.gdb and make it the workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

# allow files to be overwritten
arcpy.env.overwriteOutput = True 
#test
arcpy.env.workspace

'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\models\\..\\..\\data\\QAQC.gdb'

## Query elevation data from database
- use SQL
- may need to convert raster to point data
    - see lab 02
- resample to avoid crashing the computer
--(may already be small enough)

In [29]:
# set save filepath
savepath = os.path.join(cwd, '..','..','data')
# arcpy.management.MakeQueryLayer(savepath + "\34.123.228.238.sde", "elevation_pts", "SELECT * FROM elevation_points", "grid_code", '', "4326", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision', "DEFINE_SPATIAL_PROPERTIES", "DO_NOT_INCLUDE_M_VALUES", "DO_NOT_INCLUDE_Z_VALUES", "0 0 0 0")

arcpy.management.MakeQueryLayer(r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\34.123.228.238.sde", "elevation_pts", "SELECT * FROM elevation_points", "grid_code", '', "4326", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision', "DEFINE_SPATIAL_PROPERTIES", "DO_NOT_INCLUDE_M_VALUES", "DO_NOT_INCLUDE_Z_VALUES", "0 0 0 0")


# Copy to GDB does not yet work as expected. Use the layer above as is.
#arcpy.management.CopyFeatures("elevtion_pts", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\elevation.shp", '', None, None, None)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000837: The workspace is not the correct workspace type.
Failed to execute (MakeQueryLayer).


In [30]:
# Export Query Layer (save to local GDB)

# set save filepath
savepath = os.path.join(cwd, '..','..','data')

arcpy.conversion.ExportFeatures("elevation_pts", savepath + "elevation_pts_ExportFeatures", '', "NOT_USE_ALIAS", 'pointid "pointid" true true false 4 Long 0 10,First,#,elevation_pts,pointid,-1,-1;grid_code "grid_code" true true false 4 Long 0 10,First,#,elevation_pts,grid_code,-1,-1', None)

#arcpy.conversion.ExportFeatures("elevation_pts", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\elevation_pts_ExportFeatures", '', "NOT_USE_ALIAS", 'pointid "pointid" true true false 4 Long 0 10,First,#,elevation_pts,pointid,-1,-1;grid_code "grid_code" true true false 4 Long 0 10,First,#,elevation_pts,grid_code,-1,-1', None)

<Result 'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\models\\..\\..\\dataelevation_pts_ExportFeatures.shp'>

In [4]:
# set variable to local layer file

#TODO: Change this to relative path, save to data folder!

elevation_pts = r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\elevation_pts_ExportFeatures"

### Sample the dataset
- define appropriate sample size (with proof)
    - This is done in lab 2 where I downsample the 1m raster to 100m, which uses bilinear interpolation (nearest neighbor), then convert those rasters to point data, selecting the mean value of rasters within the point conversion grid.
    - might need to downsample again for size, but this is running well now.

### Run 3 different interpolation algorithms, save to local gdb
- ordinary Kriging
- IDW
- Empirical Bayesian Kriging
- Local Polynomial KDE

In [ ]:
# Ordinary Kriging
out_surface_raster = arcpy.sa.Kriging(elevation_pts, "grid_code", "Spherical 0.000125 # # #", 0.000124525892000065, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\Kriging_lab03")

In [15]:
# IDW
elevation_pts = r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\elevation_pts_ExportFeatures"

arcpy.ga.IDW(elevation_pts, "grid_code", "idw_stats", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\idw_lab03", 0.000124525892000065, 2, "NBRTYPE=Standard S_MAJOR=1.76434719213634E-02 S_MINOR=1.76434719213634E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", None)

idw_stats = "idw_stats"

# I'm getting the error below bc of file path stuff change to full filepath

In [23]:
# Empirical Bayesian Kriging
arcpy.ga.EmpiricalBayesianKriging(elevation_pts, "grid_code", "Bayesian_Kriging_stats", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\Bayesian_Kriging_lab03", 0.000124525892000065, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=1.76434719213634E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")
bayesian_kriging_stats = "Bayseian_kriging_stats"

In [24]:
# local polynomial kernel
arcpy.ga.LocalPolynomialInterpolation(elevation_pts, "grid_code", "lcl_polynomial_stats", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\lcl_polynomial_lab03", 0.000124525892000065, 1, "NBRTYPE=Standard S_MAJOR=1.76434719213634E-02 S_MINOR=1.76434719213634E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EXPONENTIAL", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")
lcl_poly_stats = "lcl_polynomial_stats"


In [ ]:
# Spline
# out_raster = arcpy.sa.Spline("elevation_pts", "grid_code", 0.000124525892000065, "REGULARIZED", 0.1, 12); out_raster.save(r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\Spline_lab03")

### Save the interpolated maps to my GDB and PostGIS DB
- convert to point layer and use psycopg2 to make new tables in sql remote db
- save raster to gdb should be basic

In [6]:
# set save filepath
savepath = os.path.join(cwd, '..','..','data')

In [7]:
# convert rasters to point layers

#arcpy.conversion.RasterToPoint("source.img", "c:/output/source.shp", "VALUE")
ordinaryKrg = arcpy.conversion.RasterToPoint(savepath + "Kriging_lab03", savepath + "ordinaryKrgPts", "VALUE")
empBayesKrg = arcpy.conversion.RasterToPoint(savepath + "Bayesian_Kriging_lab03",savepath + "BayesianKrgPts", "VALUE")
idw = arcpy.conversion.RasterToPoint(savepath + "idw_lab03", savepath + "idwPts", "VALUE")
lclPolyKernel = arcpy.conversion.RasterToPoint(savepath + "lcl_polynomial_lab03", savepath + "lclPolyPts", "VALUE")
with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', outputZFlag="Disabled", outputMFlag="Disabled"):
    #arcpy.conversion.RasterToPoint("idw_lab03", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\RasterT_idw_lab1", "Value")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000860: Input raster: is not the type of Composite Geodataset, or does not exist.
WARNING 001000: Field: Field VALUE does not exist
Failed to execute (RasterToPoint).


In [ ]:
# create table and upload data to sql DB

## Evaluate the interpolated estimates of each model compared to the true DEM
- leave one out CROSS VALIDATION in geoprocessing arcpy
- store this accuracy assessment as a table in postgres
- and in home gdb

## Find and store differences between ground truth and model predictions in a point layer
- visualize this like in Luke's slides
- save to proj GDB and postgres
- compute rmse?

In [ ]:
# Cross Validation, outputs a point layer with stats including error for each point

# idw
arcpy.ga.CrossValidation("idw_stats", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\idw_crossval")

In [ ]:
#Bayseian Kriging
arcpy.ga.CrossValidation("Bayesian_Kriging_stats", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\Bayesian_Kriging_crossval")

In [ ]:
# local polynomial kernel
arcpy.ga.CrossValidation("lcl_polynomial_stats", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\lcl_polynomial_crossval")

In [ ]:
# Compare results, output comparison table, table with best fit model by percent accuracy
# outputs a geostatistical comparison in the arc project
arcpy.ga.CompareGeostatisticalLayers("lcl_polynomial_stats;idw_stats;Bayesian_Kriging_stats", r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb\CompareGeostatisticalLayers1", "BestInterpByComparrison", "SINGLE", "ACCURACY", "ACCURACY PERCENT #", "ACCURACY 1", None)

## save to postgres

In [27]:
# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')

# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

try:
    conn = psycopg2.connect(**pg_connection_dict)
    print("connected")
except:
    print("connection failed")

connected


In [ ]:
#TODO make this a function, pass a series of lists with the correct tabel credentials and table names 
#TODO clean up the file path mess, it's causing all your headaches

# set save filepath
savepath = os.path.join(cwd, '..','..','data')

points = os.path.join(savepath, "aoi_elev_4326_points.shp")
fields_points = ['pointid', 'grid_code', "Shape@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS elevation_points")
cursor.execute("""
    CREATE TABLE elevation_points (
        id SERIAL,
        pointid INT,
        grid_code INT)
""")

cursor.execute("""
    SELECT AddGeometryColumn('elevation_points', 'geom', 4326, 'POINT', 2)
""")

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO elevation_points (pointid, grid_code, geom) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))

conn.commit()

In [ ]:
#dumping it into a numpy array for calculating rmse
input = r"C:\Users\laure\OneDrive\Documents\ArcGIS\Projects\PurpleAirStations\PurpleAirStations.gdb\lp_cv"
arr_temp_lp = arcpy.da.TableToNumPyArray(input, ("OBJECTID", "Error"))

#putting the numpy array into a pd df
lp_temp_arr = pd.DataFrame(arr_temp_lp, columns=["OBJECTID", "Error"])

# Calculate squared error
lp_temp_arr['squared_error'] = lp_temp_arr['Error'] ** 2

# Calculate RMSE
lp_rmse = numpy.sqrt(lp_temp_arr['squared_error'].mean())

# Display RMSE
print("RMSE: ", lp_rmse)

DO THE ABOVE FOR EACH INTERPOLATION

#bringing the RMSE values together in a pd df to compare

temp_compare = pd.DataFrame({'method':['kriging', 'idw', 'local polynomial'],
                            'RMSE':[krig_rmse, idw_rmse, lp_rmse]})
temp_compare

#conclusion
# Get index of row with lowest value of rmse
min_index = temp_compare['RMSE'].idxmin()

# Get method with lowest rmse
min_id = temp_compare.loc[min_index, 'method']

print("Method with the lowest RMSE value is", min_id)